- logistic回归
- 反向传播


## 反向传播
这应该是神经网络里面最难的部分了吧～～为此除了CS231n的课，还结合了Coursera上Ng的deeplearning课程



### logistic 回归

想直接看反向传播的视频的，但发现Ng的符号使用不太一样，所以先从头看点～顺便重新理解了一下logistic回归，貌似又有了新的感觉～就记录下来吧

**1. 符号表示：**

input：$x.shape = R^{n_x}$ 样本特征是以列的形式放置的

m training example $\{(x^{(1)},y^{(1)}),(x^{(2)},y^{(2)}),..,(x^{(n)},y^{(n)})\}$

$X\in R^{n_x*m}$ ,so $X.shape = (n_x,m)$

label：$y\in\{0,1\}$,so $y.shape = (1,m)$


**2. forward**

我们需要将计算得到的输出与真实标签y进行对比，即$\hat y = P(y=1|x)$

parameters: $W\in R^{n_x*1},b\in R$ 

output:

$z = w^T.dot(x)+b$

$\hat y = sigmoid(w^T.dot(x)+b)$ 将$\hat y 转化到0到1之间的概率$

$$sigmoid(z) = \frac{1}{1+e^{-z}}$$
![这里写图片描述](http://img.blog.csdn.net/20180202112641621?watermark/2/text/aHR0cDovL2Jsb2cuY3Nkbi5uZXQvcGFueGlhb3hpZQ==/font/5a6L5L2T/fontsize/400/fill/I0JBQkFCMA==/dissolve/70/gravity/SouthEast)


**3. cost function: **不是采用的均方差，因为得到的函数不是Convex的，至于为什么有机会看看。。总之最后是下面这样：
$$L(\hat y,y) = -ylog(\hat y)-(1-y)log(1-\hat y)$$

后面学过softmax就会发现这其实就是二分类的softmax啊～～而softmax就是多分类的logistic回归～

对单个样本来说：对应起来就是$\hat y = \frac{e^{f_{y_i}}}{\sum_j e^{f_j}}$,losistic的概率只有一个，而softmax的概率有j=class个～
至此，也发现，回归相对于分类来说就是回归能给出每一类的概率，而分类只选最好的那个。


除了可以从softmax角度来理解这个损失函数，还可以从对数似然估计来推导～～以前推过，这儿就不写了

**4. 梯度下降**

$ w =w- dw = w- \dfrac{\partial J(w,b)}{\partial w}$ ，  其中loss对权重w的梯度怎么求？这就是反向传播的重难点～～

<img src= "http://img.blog.csdn.net/20180202133635926?watermark/2/text/aHR0cDovL2Jsb2cuY3Nkbi5uZXQvcGFueGlhb3hpZQ==/font/5a6L5L2T/fontsize/400/fill/I0JBQkFCMA==/dissolve/70/gravity/SouthEast" width=500 height=500 />


对单个样本的求导就是链式法则～与以前学过的求导是一样的～



$"da" = \dfrac{\partial L}{\partial a} = -\dfrac{y}{a}+\dfrac{1-y}{1-a}$

$dz = \dfrac{dL}{dz} = \dfrac{dL}{da}\dfrac{da}{dz}=a-y$

$dw_1 =\dfrac{dL}{dz}\dfrac{dz}{dw_1}= x_1dz = x_1(a-y)$ 

$db= \dfrac{dL}{dz}\dfrac{dz}{db} = dz$

<img src="http://img.blog.csdn.net/20180202135833354?watermark/2/text/aHR0cDovL2Jsb2cuY3Nkbi5uZXQvcGFueGlhb3hpZQ==/font/5a6L5L2T/fontsize/400/fill/I0JBQkFCMA==/dissolve/70/gravity/SouthEast" width=600 height=600>

$dw_1 = \dfrac{1}{m}\sum_{i=1}^m x^i(a^i-y^i)$

$db = \dfrac{1}{m}\sum_{i=1}^m dz^i = \dfrac{1}{m}\sum_{i=1}^m (a^i-y^i) $

然后将单个样本的推广到m个样本，公式Ng写的很清楚了～～～不得不说Ng太适合教学了，由浅入深，通俗易懂，是个人都听的懂啊！！

再回到问题中来，dw在计算时用的累加器，因为权重参数W是共享的，也就是每个样本都是在同样的一组权重下求得的loss，因此每个样本都会有对应的dw，累加之后求平均值$dw_1$，就是整个数据集经过一次梯度下降$w_1$的变化量。

但这样求梯度，需要连个循环，一个是样本m的循环，一个是特征的循环(这里$X\in R^{n_x*m},W\in R^{n_x*1}$,总共有$n_x$个特征)，这样两重循环非常耗费计算资源。所以需要引入向量化的计算。

然后就是矩阵运算，有些地方表示很难理解。。。这也是看Ng视频跟着推导一遍的主要目的！

#### 5. 向量化

forward：

线性输出：$Z = W^T X+b$

非线性化后：$A = sigmoid(Z)$


backward:

$dZ = A -Y$

$dw = \frac{1}{m}XdZ^T$  

$db = \frac{1}{m}np.sum(dZ)$

<img src="http://img.blog.csdn.net/20180202151941372?watermark/2/text/aHR0cDovL2Jsb2cuY3Nkbi5uZXQvcGFueGlhb3hpZQ==/font/5a6L5L2T/fontsize/400/fill/I0JBQkFCMA==/dissolve/70/gravity/SouthEast" width=300 height=300>



### 6. 来自Ng建议

donnot use “rank 1 array” 

尽量使用reshape() 和 assert()

In [1]:
import numpy as np 

a = np.random.rand(5)

In [5]:
print(a)

[ 0.22320117  0.59663676  0.16794532  0.56259808  0.1825964 ]


In [6]:
print(a.T)

[ 0.22320117  0.59663676  0.16794532  0.56259808  0.1825964 ]


In [7]:
print(a.dot(a.T))

0.783857867461


In [9]:
b = np.random.rand(1,5)
print(b)
print(a-b)

[[ 0.19164006  0.64546438  0.78617663  0.96930788  0.99644683]]
[[ 0.03156112 -0.04882762 -0.61823131 -0.40670979 -0.81385043]]


So, rank 1 array is a strange thing~

###  反向传播

重新推导了一遍logistic的梯度之后，我们再来看看神经网络中的反向传播～